In [1]:
import parse

In [2]:
class Mask:
    mask: str = None
    and_mask: int = None
    or_mask: int = None
    def __init__(self, mask: str):
        self.mask = mask
        self.and_mask = int(mask.replace('X', '1'), 2)
        self.or_mask = int(mask.replace('X', '0'), 2)
    def apply(self, value):
        return (value & self.and_mask) | self.or_mask
    def __repr__(self):
        return f"mask = {self.mask}"
    def __str__(self):
        return repr(self)

In [3]:
class MaskV2:
    mask: str = None
    or_mask = None
    floating_masks = None
    def __init__(self, mask: str):
        self.mask = mask
        self.or_mask = int(mask.replace('X', '1'), 2)
        self.get_floating_masks()
    def get_floating_masks(self):
        self.floating_masks = []
        partials = ['']
        for bit in self.mask:
            new_partials = []
            for p in partials:
                new_partials.append(p + '1')
                if bit == 'X':
                    new_partials.append(p + '0')
            partials = new_partials
        self.floating_masks = [int(p, 2) for p in partials]
    def apply(self, value):
        value = value | self.or_mask
        values = []
        for fmask in self.floating_masks:
            values.append(value & fmask)
        return values
    def __repr__(self):
        return f"mask = {self.mask}"
    def __str__(self):
        return repr(self)

In [4]:
test_mask = Mask('XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X')

In [5]:
test_mask.apply(11)

73

In [6]:
class Assignment:
    address: int = 0
    value: int = 0
    def __init__(self, address: int, value: int):
        self.address = address
        self.value = value
    def __repr__(self):
        return f"mem[{self.address}] = {self.value}"

In [7]:
def get_input(fname="input.txt", mask_cls=Mask):
    instructions = []
    pattern = parse.compile('mem[{}] = {}')
    with open(fname) as f:
        for line in f.readlines():
            line = line.strip()
            if line[:4] == 'mask':
                instructions.append(mask_cls(line.split(' = ')[-1]))
            else:
                parsed = pattern.parse(line)
                instructions.append(Assignment(int(parsed[0]), int(parsed[1])))
    return instructions

In [8]:
test_data = get_input("test.txt")

In [9]:
test_data

[mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X,
 mem[8] = 11,
 mem[7] = 101,
 mem[8] = 0]

In [10]:
def run(instructions):
    mask = None
    values = {}
    for instr in instructions:
        if type(instr) is Mask:
            mask = instr
            continue
        values[instr.address] = mask.apply(instr.value)
    return values

In [11]:
test_result = run(test_data)

In [12]:
sum(test_result.values())

165

In [13]:
input_data = get_input("input.txt")

In [14]:
input_result = run(input_data)

In [15]:
sum(input_result.values())

6559449933360

In [16]:
max(sum(c == 'X' for c in i.mask) for i in input_data if type(i) is Mask)

9

In [17]:
mask_v2 = MaskV2('000000000000000000000000000000X1001X')

In [18]:
mask_v2.apply(42)

[59, 58, 27, 26]

In [19]:
test2_data = get_input("test2.txt", MaskV2)

In [20]:
def run2(instructions):
    mask = None
    values = {}
    for instr in instructions:
        if type(instr) is MaskV2:
            mask = instr
            continue
        for addr in mask.apply(instr.address):
            values[addr] = instr.value
    return values

In [21]:
sum(run2(test2_data).values())

208

In [22]:
input2_data = get_input("input.txt", MaskV2)

In [23]:
sum(run2(input2_data).values())

3369767240513